In [5]:
import tkinter as tk
from tkinter import ttk, messagebox

class CineMatch:
    def __init__(self):
        self.movies = {}
        self.sorted_movies = []

    def add_movie(self, title, genre, rating, actors, year, industry, ott_platform):
        if title in self.movies:
            return False
        self.movies[title] = {
            'genre': genre,
            'rating': rating,
            'actors': actors,
            'year': year,
            'industry': industry,
            'ott_platform': ott_platform
        }
        self.sorted_movies.append((title, rating))
        self.sorted_movies = self.merge_sort(self.sorted_movies)
        return True

    def merge_sort(self, arr):
        if len(arr) > 1:
            mid = len(arr) // 2
            left_half = arr[:mid]
            right_half = arr[mid:]

            self.merge_sort(left_half)
            self.merge_sort(right_half)

            i = j = k = 0

            while i < len(left_half) and j < len(right_half):
                if left_half[i][1] > right_half[j][1]:
                    arr[k] = left_half[i]
                    i += 1
                else:
                    arr[k] = right_half[j]
                    j += 1
                k += 1

            while i < len(left_half):
                arr[k] = left_half[i]
                i += 1
                k += 1

            while j < len(right_half):
                arr[k] = right_half[j]
                j += 1
                k += 1

        return arr

    def search_movies(self, search_term, by='title'):
        results = []
        if by == 'title':
            results = [title for title in self.movies if search_term.lower() in title.lower()]
        elif by == 'genre':
            results = [title for title, details in self.movies.items() if search_term.lower() in details['genre'].lower()]
        elif by == 'actor':
            results = [title for title, details in self.movies.items() if any(search_term.lower() in actor.lower() for actor in details['actors'])]
        elif by == 'industry':
            results = [title for title, details in self.movies.items() if search_term.lower() in details['industry'].lower()]
        return results

    def recommend_top_n(self, n, by='rating'):
        if by == 'rating':
            return self.sorted_movies[:n]
        elif by == 'industry':
            industry_dict = {}
            for title, details in self.movies.items():
                if details['industry'] not in industry_dict:
                    industry_dict[details['industry']] = []
                industry_dict[details['industry']].append((title, details['rating']))
            recommendations = []
            for industry, movies in industry_dict.items():
                movies = self.merge_sort(movies)
                recommendations.extend(movies[:n])
            return recommendations[:n]

    def delete_movie(self, title):
        if title not in self.movies:
            return False
        del self.movies[title]
        self.sorted_movies = [movie for movie in self.sorted_movies if movie[0] != title]
        return True

class CineMatchGUI:
    def __init__(self, root, cine_match):
        self.root = root
        self.cine_match = cine_match
        self.root.title("CineMatch")

        self.tab_control = ttk.Notebook(root)
        
        self.add_tab = ttk.Frame(self.tab_control)
        self.search_tab = ttk.Frame(self.tab_control)
        self.recommend_tab = ttk.Frame(self.tab_control)
        self.delete_tab = ttk.Frame(self.tab_control)

        self.tab_control.add(self.add_tab, text="Add Movie")
        self.tab_control.add(self.search_tab, text="Search Movie")
        self.tab_control.add(self.recommend_tab, text="Recommend Movies")
        self.tab_control.add(self.delete_tab, text="Delete Movie")
        self.tab_control.pack(expand=1, fill="both")

        self.create_add_tab()
        self.create_search_tab()
        self.create_recommend_tab()
        self.create_delete_tab()

    def create_add_tab(self):
        self.add_frame = ttk.LabelFrame(self.add_tab, text="Add a New Movie")
        self.add_frame.pack(padx=10, pady=10, fill="both", expand=True)

        ttk.Label(self.add_frame, text="Title:").grid(column=0, row=0, padx=5, pady=5)
        self.title_entry = ttk.Entry(self.add_frame)
        self.title_entry.grid(column=1, row=0, padx=5, pady=5)

        ttk.Label(self.add_frame, text="Genre:").grid(column=0, row=1, padx=5, pady=5)
        self.genre_combo = ttk.Combobox(self.add_frame, values=["Action", "Drama", "Comedy", "Sci-Fiction", "Horror", "Fantasy"])
        self.genre_combo.grid(column=1, row=1, padx=5, pady=5)
        self.genre_combo.current(0)

        ttk.Label(self.add_frame, text="Rating:").grid(column=0, row=2, padx=5, pady=5)
        self.rating_entry = ttk.Entry(self.add_frame)
        self.rating_entry.grid(column=1, row=2, padx=5, pady=5)

        ttk.Label(self.add_frame, text="Actors:").grid(column=0, row=3, padx=5, pady=5)
        self.actors_entry = ttk.Entry(self.add_frame)
        self.actors_entry.grid(column=1, row=3, padx=5, pady=5)

        ttk.Label(self.add_frame, text="Year:").grid(column=0, row=4, padx=5, pady=5)
        self.year_entry = ttk.Entry(self.add_frame)
        self.year_entry.grid(column=1, row=4, padx=5, pady=5)

        ttk.Label(self.add_frame, text="Industry:").grid(column=0, row=5, padx=5, pady=5)
        self.industry_entry = ttk.Entry(self.add_frame)
        self.industry_entry.grid(column=1, row=5, padx=5, pady=5)

        ttk.Label(self.add_frame, text="OTT Platform:").grid(column=0, row=6, padx=5, pady=5)
        self.ott_entry = ttk.Entry(self.add_frame)
        self.ott_entry.grid(column=1, row=6, padx=5, pady=5)

        self.add_button = ttk.Button(self.add_frame, text="Add Movie", command=self.add_movie)
        self.add_button.grid(column=0, row=7, columnspan=2, padx=5, pady=10)

        ttk.Label(self.add_frame, text="Movies Added:").grid(column=0, row=8, columnspan=2, padx=5, pady=5)
        self.movies_list = tk.Text(self.add_frame, height=10, width=50)
        self.movies_list.grid(column=0, row=9, columnspan=2, padx=5, pady=5)

    def add_movie(self):
        title = self.title_entry.get()
        genre = self.genre_combo.get()
        try:
            rating = float(self.rating_entry.get())
            actors = self.actors_entry.get().split(',')
            year = int(self.year_entry.get())
            industry = self.industry_entry.get()
            ott_platform = self.ott_entry.get()

            if self.cine_match.add_movie(title, genre, rating, actors, year, industry, ott_platform):
                messagebox.showinfo("Success", "Movie added successfully!")
                self.update_movies_list()
                self.clear_entries()
            else:
                messagebox.showerror("Error", "Movie already exists.")
        except ValueError:
            messagebox.showerror("Error", "Invalid input for rating or year.")

    def update_movies_list(self):
        self.movies_list.delete(1.0, tk.END)
        for title in self.cine_match.movies:
            self.movies_list.insert(tk.END, f"{title}\n")

    def clear_entries(self):
        self.title_entry.delete(0, tk.END)
        self.genre_combo.set('Action')
        self.rating_entry.delete(0, tk.END)
        self.actors_entry.delete(0, tk.END)
        self.year_entry.delete(0, tk.END)
        self.industry_entry.delete(0, tk.END)
        self.ott_entry.delete(0, tk.END)

    def create_search_tab(self):
        self.search_frame = ttk.LabelFrame(self.search_tab, text="Search Movies")
        self.search_frame.pack(padx=10, pady=10, fill="both", expand=True)

        self.search_term_label = ttk.Label(self.search_frame, text="Search Term:")
        self.search_term_label.grid(column=0, row=0, padx=5, pady=5)
        self.search_term_entry = ttk.Entry(self.search_frame)
        self.search_term_entry.grid(column=1, row=0, padx=5, pady=5)

        self.search_by_label = ttk.Label(self.search_frame, text="Search By:")
        self.search_by_label.grid(column=0, row=1, padx=5, pady=5)
        self.search_by_combo = ttk.Combobox(self.search_frame, values=["title", "genre", "actor", "industry"])
        self.search_by_combo.grid(column=1, row=1, padx=5, pady=5)
        self.search_by_combo.current(0)

        self.search_button = ttk.Button(self.search_frame, text="Search", command=self.search_movies)
        self.search_button.grid(column=0, row=2, columnspan=2, padx=5, pady=10)

        self.search_results = tk.Text(self.search_frame, height=10)
        self.search_results.grid(column=0, row=3, columnspan=2, padx=5, pady=5)

    def search_movies(self):
        search_term = self.search_term_entry.get()
        search_by = self.search_by_combo.get()
        results = self.cine_match.search_movies(search_term, by=search_by)
        self.search_results.delete(1.0, tk.END)
        if results:
            for result in results:
                self.search_results.insert(tk.END, result + "\n")
        else:
            self.search_results.insert(tk.END, "No movies found.")

    def create_recommend_tab(self):
        self.recommend_frame = ttk.LabelFrame(self.recommend_tab, text="Recommend Top N Movies")
        self.recommend_frame.pack(padx=10, pady=10, fill="both", expand=True)

        ttk.Label(self.recommend_frame, text="Number of Movies:").grid(column=0, row=0, padx=5, pady=5)
        self.num_movies_entry = ttk.Entry(self.recommend_frame)
        self.num_movies_entry.grid(column=1, row=0, padx=5, pady=5)

        ttk.Label(self.recommend_frame, text="Recommend By:").grid(column=0, row=1, padx=5, pady=5)
        self.recommend_by_combo = ttk.Combobox(self.recommend_frame, values=["rating", "industry"])
        self.recommend_by_combo.grid(column=1, row=1, padx=5, pady=5)
        self.recommend_by_combo.current(0)

        self.recommend_button = ttk.Button(self.recommend_frame, text="Recommend", command=self.recommend_movies)
        self.recommend_button.grid(column=0, row=2, columnspan=2, padx=5, pady=10)

        self.recommend_results = tk.Text(self.recommend_frame, height=10)
        self.recommend_results.grid(column=0, row=3, columnspan=2, padx=5, pady=5)

    def recommend_movies(self):
        try:
            num_movies = int(self.num_movies_entry.get())
            recommend_by = self.recommend_by_combo.get()
            recommendations = self.cine_match.recommend_top_n(num_movies, by=recommend_by)
            self.recommend_results.delete(1.0, tk.END)
            if recommendations:
                for title, rating in recommendations:
                    self.recommend_results.insert(tk.END, f"{title} (Rating: {rating})\n")
            else:
                self.recommend_results.insert(tk.END, "No recommendations found.")
        except ValueError:
            messagebox.showerror("Error", "Please enter a valid number for 'Number of Movies'.")

    def create_delete_tab(self):
        self.delete_frame = ttk.LabelFrame(self.delete_tab, text="Delete a Movie")
        self.delete_frame.pack(padx=10, pady=10, fill="both", expand=True)

        ttk.Label(self.delete_frame, text="Title:").grid(column=0, row=0, padx=5, pady=5)
        self.delete_title_entry = ttk.Entry(self.delete_frame)
        self.delete_title_entry.grid(column=1, row=0, padx=5, pady=5)

        self.delete_button = ttk.Button(self.delete_frame, text="Delete Movie", command=self.delete_movie)
        self.delete_button.grid(column=0, row=1, columnspan=2, padx=5, pady=10)

    def delete_movie(self):
        title = self.delete_title_entry.get()
        if self.cine_match.delete_movie(title):
            messagebox.showinfo("Success", "Movie deleted successfully!")
            self.update_movies_list()
        else:
            messagebox.showerror("Error", "Movie not found.")

if __name__ == "__main__":
    root = tk.Tk()
    cine_match = CineMatch()
    app = CineMatchGUI(root, cine_match)
    root.mainloop()
